<a href="https://colab.research.google.com/github/IMI1998/pyTorch-LSTM-/blob/main/pyTorch(LSTM).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
import argparse
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets , transforms
from torch.optim.lr_scheduler import StepLR

In [43]:
class Net(nn.Module):
  def __init__(self):
    super(Net,self).__init__()
    self.rnn = nn.LSTM(input_size=28 , hidden_size=64 , batch_first=True)
    self.batchnorm = nn.BatchNorm1d(64)
    self.dropout1 = nn.Dropout2d(0.25)
    self.dropout2 = nn.Dropout2d(0.5)
    self.fc1 = nn.Linear(64,32)
    self.fc2 = nn.Linear(32,10)

  def forward(self , input):
    input = input.reshape(-1 , 28 , 28)
    output , hidden = self.rnn(input)
    output = output[: , -1 , :]
    output = self.batchnorm(output)
    output = self.dropout1(output)
    output = self.fc1(output)
    output = F.relu(output)
    output = self.dropout2(output)
    output = self.fc2(output)
    output = F.log_softmax(output , dim=1)
    return output






In [44]:
def train(model , device , train_loader , optimizer , epoch):
  model.train()
  for batch_idx , (data , target) in enumerate(train_loader):
    data , target = data.to(device) , target.to(device)
    optimizer.zero_grad()
    output = model(data)
    loss = F.nll_loss(output , target)
    loss.backward()
    optimizer.step()
    if batch_idx % 10 == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)] \t loss: {:.6f}'.format(
          epoch , batch_idx * len(data) , len(train_loader.dataset),
          100. * batch_idx / len (train_loader) , loss.item()
      ))


In [45]:
def test(model , device , test_loader):
  model.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data , target in test_loader:
      data , target = data.to(device) , target.to(device)
      output = model(data)
      test_loss += F.nll_loss(output , target , reduction='sum').item()
      pred = output.argmax(dim=1 , keepdim=True)
      correct += pred.eq(target.view_as(pred)).sum().item()

  test_loss /= len(test_loader.dataset)

  print('\n Test set: Average loss: {:.4f} , Accuracy: {}/{} ({:.0f}%)\n' .format(
      test_loss , correct , len(test_loader.dataset), 100. * correct / len(test_loader.dataset)
  ))

In [46]:
use_cuda = torch.cuda.is_available()


torch.manual_seed(45123)

device = torch.device('cuda' if use_cuda else 'cpu')

kwargs = {'num_workers' : 1 ,'pin_memory' : True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data' , train=True , download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((.1307,) , (0.3081,))
                    ])),
     batch_size=64 , shuffle=True , **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data' , train=False , transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((.1307,) , (0.3081,))
                    ])),
    batch_size=1000 , shuffle=True , **kwargs)




model = Net().to(device)
optimizer = optim.Adadelta(model.parameters() , lr=1.0)

scheduler = StepLR(optimizer , step_size=1 , gamma=0.7)
for epoch in range(1,10):
    train(model , device , train_loader , optimizer , epoch)
    test(model , device , test_loader)
    scheduler.step()

torch.save(model.state_dict() , "mnist_rnn.pt")

Train Epoch: 1 [0/60000 (0%)] 	 loss: 2.379921
Train Epoch: 1 [640/60000 (1%)] 	 loss: 2.023040
Train Epoch: 1 [1280/60000 (2%)] 	 loss: 1.994041
Train Epoch: 1 [1920/60000 (3%)] 	 loss: 1.686792
Train Epoch: 1 [2560/60000 (4%)] 	 loss: 1.666762
Train Epoch: 1 [3200/60000 (5%)] 	 loss: 1.599903
Train Epoch: 1 [3840/60000 (6%)] 	 loss: 1.505366
Train Epoch: 1 [4480/60000 (7%)] 	 loss: 1.404593
Train Epoch: 1 [5120/60000 (9%)] 	 loss: 1.370061
Train Epoch: 1 [5760/60000 (10%)] 	 loss: 1.385309
Train Epoch: 1 [6400/60000 (11%)] 	 loss: 1.273804
Train Epoch: 1 [7040/60000 (12%)] 	 loss: 1.050079
Train Epoch: 1 [7680/60000 (13%)] 	 loss: 1.135807
Train Epoch: 1 [8320/60000 (14%)] 	 loss: 1.115175
Train Epoch: 1 [8960/60000 (15%)] 	 loss: 1.063548
Train Epoch: 1 [9600/60000 (16%)] 	 loss: 0.788229
Train Epoch: 1 [10240/60000 (17%)] 	 loss: 1.027339
Train Epoch: 1 [10880/60000 (18%)] 	 loss: 1.049800
Train Epoch: 1 [11520/60000 (19%)] 	 loss: 0.899521
Train Epoch: 1 [12160/60000 (20%)] 	 loss